In [ ]:
import networkx as nx
import heapq

In [ ]:
G = nx.DiGraph()
edges = [(1, 2, 10),
         (2, 3, 1),
         (3, 4, 10),
         (1, 9, 20),
         (9, 2, 1),
         (2, 6, 1),
         (6, 8, 1),
         (8, 9, 1),
         (2, 8, 3),
         (8, 5, 15),
         (3, 5, 18),
         (5, 4, 1)]

# A 1
# B 2
# C 3
# D 4
# E 5
# F 6
# G 7
# H 8
# I 9

G.add_weighted_edges_from(edges)

In [ ]:
nx.draw(G, with_labels=True)

In [ ]:
# partial dijkstra
def dijkstra(graph, src, dest):

  if src ==  dest:
    return 0

  heap = [(0, src)]
  shortest_path = {node: float('inf') for node in graph}
  shortest_path[src] = 0

  while heap:
    cost, node = heapq.heappop(heap)
    if node == dest:
      # print(shortest_path)
      return shortest_path[dest]

    if cost > shortest_path[node]:
      continue

    for neighbor, data in graph[node].items():
      new_cost = cost + data['weight']

      if new_cost < shortest_path[neighbor]:
        shortest_path[neighbor] = new_cost
        heapq.heappush(heap, (new_cost, neighbor))

  return float('inf')
 

In [ ]:
def reverse(graph):
  Gr = nx.DiGraph()
  # for node in graph.nodes():
  #   Gr.add_node(node)

  Gr.add_edges_from((v,u,d) for u,v,d in graph.edges(data=True))
  return Gr

In [ ]:
GR = reverse(G)

In [ ]:
nx.draw(GR, with_labels=True)

In [ ]:
dist = {}
isSettled = {}
PQ = []
parent = {}

for node in GR.nodes():
  dist[node] = float('inf')
  isSettled[node] = False
  parent[node] = None

dest = 4 # D vertexi destination noktamız
heapq.heappush(PQ, (0, dest))
dist[dest] = 0

In [ ]:
def ConstructPartialSPT(graph, v):
  global dist, isSettled, PQ, parent

  if isSettled[v]:
    return dist[v]

  while PQ:
    cost, node = heapq.heappop(PQ)

    if cost > dist[node]:
      continue

    if not isSettled[node]:
      isSettled[node] = True

      for neighbor, data in graph[node].items():

        if not isSettled[neighbor]:        
          new_cost = cost + data['weight']

          if new_cost < dist[neighbor]:
            dist[neighbor] = new_cost
            parent[neighbor] = node
            heapq.heappush(PQ, (new_cost, neighbor))

      if node == v:
        return dist[v]      
      
  return float('inf')

In [ ]:
ConstructPartialSPT(GR, 5)
print(dist)
print(isSettled)
print(parent)

In [ ]:
ConstructPartialSPT(GR, 2)
print(dist)
print(isSettled)
print(parent)

In [ ]:
ConstructPartialSPT(GR, 1)
print(dist)
print(isSettled)
print(parent)

In [ ]:
def LB1(length, node):
  return length + dist[node] 


In [ ]:
result = [{(1,2): 10,
           (2,3): 1,
           (3, 4): 10}]

In [ ]:
# path dict olarak tutuluyor (edgelerin listesi)
# result dict arrayi olarak tutuluyor
def LB2(new_path, threshold):
  global result
  lb2 = 0
  for old_path in result: 

    common_edges = set(old_path.keys()).intersection(set(new_path.keys()))

    intersection_length = sum(old_path[e] for e in common_edges)
    # print(intersection_length)
    old_path_length = sum(e for e in old_path.values())
    # print(old_path_length)

    current_lb2 = intersection_length * (1+1/threshold) - old_path_length
    lb2 = max(lb2, current_lb2)

  #   if old_path in new_path:
  #     s += result[old_path]

  # print(s)
  # ss = sum(result.values())
  return lb2


In [ ]:

path = {(1,2): 10,
        (2,6): 1,
        (6, 7): 1,
        (7, 5): 15,
        (5, 4): 1}

LB2(path, 0.5)

In [ ]:
def lb(LB1, LB2):
  return max(LB1, LB2)

In [ ]:
def Sim(new_path, threshold):
  flag = True

  for old_path in result:
    common_edges = set(new_path.keys()).intersection(set(old_path.keys()))
    intersection_length = sum(new_path[e] for e in common_edges)

    union_edges = set(new_path.keys()).union(set(old_path.keys()))
    union_length = sum(new_path[e] for e in union_edges if e in new_path)
    union_length += sum(old_path[e] for e in union_edges if e in old_path)
    union_length -= intersection_length


    similarity = intersection_length / union_length

    if similarity >= threshold:
      flag = False
      break  

  return flag

In [ ]:
Sim(path, 0.5)

In [ ]:
# path dönen dijkstra
def dijkstra3(graph, src, dest):

  if src == dest:
    return {}, 0

  heap = [(0, src, [])]
  visited = set()

  while heap:
    cost, node, path = heapq.heappop(heap)

    if node in visited:
      continue
    visited.add(node)

    if node == dest:
      shortest_path = {}

      for i in range(len(path)):
        if i < len(path) - 1:
          u, v = path[i], path[i+1]
          shortest_path[(u,v)] = graph[u][v]['weight']
        else:
          u, v = path[i], dest
          shortest_path[(u,v)] = graph[u][v]['weight']

      return shortest_path

    for neighbor, data, in graph[node].items():
      if neighbor not in visited:
        new_cost = cost + data['weight']
        heapq.heappush(heap, (new_cost, neighbor, path + [node]))

  return None, float('inf')